In [132]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
import string
import numpy as np

In [133]:
census = pd.read_csv('MA_CensusTract_Zipcode_merge.csv')

In [134]:
tracts = census['POSTCODE'].unique().tolist()
tractsZips = dict.fromkeys(tracts)

In [135]:
groups = census.groupby('POSTCODE')
for POSTCODE, frame in groups:
    tractsZips[POSTCODE] = frame['NAME10'].values.tolist()

In [136]:
erma = pd.read_csv('ERMA Payments 7.1.20-3.22.21.csv')
erma['PAY_TYPE'] = 'ERMA'
erma['CETRACT'] = ''

raft = pd.read_csv('RAFT Payments 1.1.19-3.22.21.xlsx - Sheet1.csv')
raft['PAY_TYPE'] = 'RAFT'
raft['CETRACT'] = ''

In [137]:
for i,row in erma.iterrows():
    val = erma.at[i, 'Unit ZIP']
    if (erma.at[i,'Unit ZIP'] in tractsZips):
        erma.at[i,'CETRACT'] = tractsZips[val]
    else:
        erma.at[i,'CETRACT'] = np.nan

In [138]:
for i,row in raft.iterrows():
    val = raft.at[i, 'Unit ZIP']
    if (raft.at[i,'Unit ZIP'] in tractsZips):
        raft.at[i,'CETRACT'] = tractsZips[val]
    else:
        raft.at[i,'CETRACT'] = np.nan

In [139]:
erma

,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT
0,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,"[7284.0, 7301.0, 7283.0, 7291.0, 7307.0, 7281...."
1,T82460,7/24/2020,"($2,857.37)",1520.0,ERMA,"[7284.0, 7301.0, 7283.0, 7291.0, 7307.0, 7281...."
2,1500022515,7/29/2020,"$3,250.00",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]"
3,1500022515,7/29/2020,"($3,250.00)",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]"
4,1500022515,7/30/2020,"($3,250.00)",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]"
...,...,...,...,...,...,...
1372,T98506,3/19/2021,"$8,577.00",2124.0,ERMA,"[9803.0, 1008.0, 1006.01, 1005.0, 1004.0, 1003..."
1373,T97039,3/19/2021,"$1,450.00",2215.0,ERMA,"[4008.0, 4003.0, 4002.0, 4001.0, 104.05, 102.0..."
1374,T97039,3/19/2021,"$1,450.00",2215.0,ERMA,"[4008.0, 4003.0, 4002.0, 4001.0, 104.05, 102.0..."
1375,T98340,3/19/2021,"$4,500.00",2119.0,ERMA,"[9803.0, 1205.0, 812.0, 806.01, 805.0, 804.01,..."


In [140]:
erma = erma.append(raft)
erma

,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT,Key Tenant Key
0,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,"[7284.0, 7301.0, 7283.0, 7291.0, 7307.0, 7281....",NaN
1,T82460,7/24/2020,"($2,857.37)",1520.0,ERMA,"[7284.0, 7301.0, 7283.0, 7291.0, 7307.0, 7281....",NaN
2,1500022515,7/29/2020,"$3,250.00",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]",NaN
3,1500022515,7/29/2020,"($3,250.00)",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]",NaN
4,1500022515,7/30/2020,"($3,250.00)",1850.0,ERMA,"[3102.0, 3103.0, 3104.0, 3143.01]",NaN
...,...,...,...,...,...,...,...
49741,1500023174,03/01/2021,-425,2639.0,RAFT,"[110.02, 117.0, 116.0, 115.0]",59486.0
49742,T8467,03/15/2021,-9100,1841.0,RAFT,"[2524.0, 2523.0, 2521.01, 2526.01, 2525.01, 25...",8467.0
49743,T97072,03/15/2021,-1350,1841.0,RAFT,"[2524.0, 2523.0, 2521.01, 2526.01, 2525.01, 25...",97072.0
49744,T97072,03/15/2021,-1350,1841.0,RAFT,"[2524.0, 2523.0, 2521.01, 2526.01, 2525.01, 25...",97072.0


In [141]:
erma.to_csv('ermatest.csv')